<h1 style="text-align:center; color:#005bbd; font-size:20px; font-family:Sans-serif; font-style: oblique; text-shadow: 0 0 3px white, 0 0 1px Black;">
   wraps Functions
</h1>

<body style="font-family: Sans-serif;">
    <div style="color: black; font-size: 15px; font-style: oblique; text-shadow: 0 0 3px white, 0 0 1px black; padding: 20px;">

**Functools.wraps** ek decorator hai jo aapke wrapper function ko original function ke metadata (jaise ke docstring, function name, aur module information) copy karne mein madad karta hai.  
  
**Bina wraps ke, problem ye hoti hai:**  
- Jab aap koi decorator use karte hain, toh wrapper function original function ko wrap kar leta hai.  
- Is se original function ka **docstring, name, aur dusri attributes lost ho jati hain** aur wrapper function ka metadata show hota hai.  
- Yeh debugging, documentation, aur introspection mein problem create karta hai.

In [1]:
def my_decorator(func):
    
    def wrapper(*args, **kwargs):
        """This is my decorator"""
        
        result = func(*args, **kwargs)
        return result
    
    return wrapper

In [2]:
@my_decorator
def greet(name):
    """This is greet function"""
    print(f"Hello, {name}!")

In [3]:
greet("Mubeen")

Hello, Mubeen!


<body style="font-family: Sans-serif;">
    <div style="color: black; font-size: 15px; font-style: oblique; text-shadow: 0 0 3px white, 0 0 1px black; padding: 20px;">
.__name__ current function ka name check karny k lye use kia gia hy

In [6]:
greet.__name__

'wrapper'

In [7]:
greet.__doc__

'This is my decorator'

<body style="font-family: Sans-serif;">
    <div style="color: black; font-size: 15px; font-style: oblique; text-shadow: 0 0 3px white, 0 0 1px black; padding: 20px;">
        
#### Solution: **functools.wraps ka use**

In [3]:
from functools import wraps

In [18]:
def my_decorator(func):
    
    @wraps(func) # Yeh decorator original function ke metadata copy karega.
    
    def wrapper(*args, **kwargs):
        """This is my decorator"""
        
        result = func(*args, **kwargs)
        return result
    
    return wrapper

In [24]:
@my_decorator
def greet(name):
    """This is greet function"""
    print(f"Hello, {name}!")

In [25]:
greet("Mubeen")

Hello, Mubeen!


In [26]:
greet.__name__

'greet'

In [27]:
greet.__doc__

'This is greet function'

<h1 style="text-align:center; color:#005bbd; font-size:20px; font-family:Sans-serif; font-style: oblique; text-shadow: 0 0 3px white, 0 0 1px Black;">
   singledispatch Functions
</h1>

<body style="font-family: Sans-serif;">
    <div style="color: black; font-size: 15px; font-style: oblique; text-shadow: 0 0 3px white, 0 0 1px black; padding: 20px;">
        
`functools.singledispatch` Python ka ek powerful tool hai jo aapko **generic functions** bananay ki sahulat deta hai. 

Iska matlab hai ke aap ek single function define kar sakte hain jo different types ke liye different behavior show kare—matlab type ke basis par function overloading. 

Yeh feature Python mein built-in function overloading ko simulate karta hai, kyunki Python mein traditionally overloading supported nahin hota.

---

#### **Why Use `singledispatch`? (Istemaal kyun karte hain?)**

1. **Code Organization aur Reusability:**
   - Aap ek generic function likh sakte hain jo multiple types handle kare.
   - Alag alag type-specific implementations ko ek hi function ke through organize kiya ja sakta hai.<br><br>

2. **Maintainability:**
   - Har type ke liye alag function likhne ki bajaye, aap base function define karke usko extend kar sakte hain.
   - Code ko modular aur asaan maintain karna hota hai.<br><br>

3. **Polymorphism:**
   - Function behavior ko type ke hisaab se change kar sakte hain bina function ke naam badle.


In [55]:
from functools import singledispatch

In [44]:
@singledispatch
def process_data(data):
    return f"Cannot process data of type {type(data)}"

<body style="font-family: Sans-serif;">
    <div style="color: black; font-size: 15px; font-style: oblique; text-shadow: 0 0 3px white, 0 0 1px black; padding: 20px;"> for Int

In [45]:
@process_data.register(int)
def _(data):
    print(f"Processing integer: {data}")
    return data * 2

<body style="font-family: Sans-serif;">
    <div style="color: black; font-size: 15px; font-style: oblique; text-shadow: 0 0 3px white, 0 0 1px black; padding: 20px;"> for string

In [47]:
@process_data.register(str)
def _(data):
    print(f"Processing string: {data}")
    return data.upper()

<body style="font-family: Sans-serif;">
    <div style="color: black; font-size: 15px; font-style: oblique; text-shadow: 0 0 3px white, 0 0 1px black; padding: 20px;"> for list

In [48]:
@process_data.register(list)
def _(data):
    print(f"Processing list with {len(data)} elements.")
    return [x * 2 for x in data]

In [49]:
process_data(10) # int

Processing integer: 10


20

In [50]:
process_data("mubeen") # string

Processing string: mubeen


'MUBEEN'

In [51]:
process_data([1,2,3]) # list

Processing list with 3 elements.


[2, 4, 6]

<body style="font-family: Sans-serif;">
    <div style="color: black; font-size: 15px; font-style: oblique; text-shadow: 0 0 3px white, 0 0 1px black; padding: 20px;"> Example 2

In [58]:
import json

@singledispatch
def serialize(obj):
    return f"Serialization not supported for type {type(obj)}"

In [59]:
@serialize.register(dict)
def _(obj: dict):
    print("Serializing dictionary")
    return json.dumps(obj)

In [60]:
@serialize.register(list)
def _(obj: list):
    print("Serializing list")
    return json.dumps(obj)

In [61]:
@serialize.register(int)
def _(obj: int):
    print("Serializing integer")
    return str(obj)

In [63]:
# Custom class ke liye serializer.
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
@serialize.register(Person)
def _(obj: Person):
    print("Serializing Person object")
    return json.dumps({"name": obj.name, "age": obj.age})

In [65]:
serialize([1, 2, 3, 4])

Serializing list


'[1, 2, 3, 4]'

In [ ]:
serialize({"key": "value", "num": 42})

In [66]:
serialize(123)

Serializing integer


'123'

In [67]:
serialize(Person("Ali", 30))

Serializing Person object


'{"name": "Ali", "age": 30}'

<h1 style="text-align:center; color:#005bbd; font-size:20px; font-family:Sans-serif; font-style: oblique; text-shadow: 0 0 3px white, 0 0 1px Black;">
   lru_cache Functions
</h1>

<body style="font-family: Sans-serif;">
    <div style="color: black; font-size: 15px; font-style: oblique; text-shadow: 0 0 3px white, 0 0 1px black; padding: 20px;"> 
        
`functools.lru_cache` ek decorator hai jo function ke results ko cache (yaani temporarily store) kar leta hai.

Is se agar aap same arguments ke saath function ko dobara call karte hain, to function dobara calculation nahi karta, balkay cached result return kar deta hai. 
        
Yeh feature khas taur pe un functions ke liye bohot useful hai jin ki calculations expensive  hoti hain aur jinhen bar bar same input ke saath call kiya jata hai.

### **Key Points:**

- **LRU ka matlab hai "Least Recently Used":**  
  Yeh cache mechanism aisa hai ke agar cache full ho jaye (maxsize set ho), to sabse pehle un entries ko remove karta hai jo sab se kam use hui hain.
  
- **Performance Improvement:**  
  Agar koi function expensive calculation karta hai (jaise recursive functions, database queries, ya heavy computation), to caching se execution time significantly reduce ho sakta hai.
  
- **Usage:**  
  `@functools.lru_cache(maxsize=None)` se aap cache ka size specify kar sakte hain. Agar `maxsize` ko None set kar dein, to cache size unlimited ho jayegi.


In [4]:
from functools import lru_cache
import time

In [27]:
@lru_cache(maxsize=128)
def heavy_computation(n):
    print(f"Performing heavy computation for n = {n}")
    time.sleep(2)  
    return sum(i * i for i in range(n))

In [28]:
start_time = time.time()
result1 = heavy_computation(100000)
end_time = time.time()
print("First call result:", result1)
print("Time taken for first call:", end_time - start_time, "seconds\n")

Performing heavy computation for n = 100000
First call result: 333328333350000
Time taken for first call: 2.008514404296875 seconds



In [30]:
start_time = time.time()
result2 = heavy_computation(10000)
end_time = time.time()
print("Second call result:", result2)
print("Time taken for second call:", end_time - start_time, "seconds")

Second call result: 333283335000
Time taken for second call: 0.00020003318786621094 seconds


<body style="font-family: Sans-serif;">
    <div style="color: black; font-size: 15px; font-style: oblique; text-shadow: 0 0 3px white, 0 0 1px black; padding: 20px;"> 
        
1. **Inside Function:**  
   - `heavy_computation(n)` ek function hai jo heavy calculation perform karta hai.  
   - Is mein `time.sleep(2)` ka use karke 2 seconds ka delay simulate kiya gaya hai, jo represent karta hai ke computation expensive hai.  
   - Function n tak ke numbers ke squares ka sum calculate karta hai.<br><br>

2. **Caching with lru_cache:**  
   - `@functools.lru_cache(maxsize=128)` decorator se function ke results cache ho jate hain.  
   - Agar same `n` ke sath function dobara call ho, to expensive computation repeat nahi hogi; balkay cached result turant return ho jayega.<br><br>

3. **Performance Improvement:**  
   - Pehli call `heavy_computation(10000)` mein 2 seconds ka delay hota hai, kyunki computation actual perform hoti hai.  
   - Second call mein same argument ke sath function jaldi execute ho jata hai kyunki result cache mein available hota hai, aur delay bohat kam (almost zero) hota hai.

<body style="font-family: Sans-serif;">
    <div style="color: black; font-size: 15px; font-style: oblique; text-shadow: 0 0 3px white, 0 0 1px black; padding: 20px;"> 
        
### **Advantage of lru_cache** 

1. **Expensive, Pure Computations:**  
   - Aise functions jinka output sirf unke input arguments pe depend karta hai aur jismein koi side effects (jaise file I/O, network calls, ya database updates) nahi hotay.
   - Misal ke taur pe, recursive algorithms (jaise Fibonacci), complex mathematical computations, ya koi heavy calculation jismein bar bar same input pe same result milta hai.

2. **Repeated Calls with Same Inputs:**  
   - Jab function ko baar baar same input ke saath call kiya jata hai, aur har baar woh expensive computation perform kar raha ho.
   - Caching se result ek dafa calculate hone ke baad fast return hota hai, jis se performance improve hoti hai.

---
        
### **Disadvantage of Lru_cache:**

1. **Functions with Side Effects:**  
   - Agar function external state change karta hai (jaise file write, network request, database update) ya koi real-time data fetch karta hai, to caching se expected behavior disturb ho sakta hai.<br><br>

2. **Dynamic/Time-Varying Results:**  
   - Jab function ka result sirf inputs pe depend nahi karta, balkay environment, time, ya kisi external resource se bhi mutasir hota hai.
   - Misal ke taur pe, sensor readings, live API responses, ya koi aisa function jo continuously update hota rahta hai.<br><br>

3. **Mutable Inputs:**  
   - Agar function mutable objects ke sath kaam karta hai aur in objects mein changes aate rehte hain, to caching se inconsistent ya outdated results mil sakte hain.